In [36]:
# # the imports in this cell are required when running on local device
# import os, sys
# sys.path.append(os.path.join('..', '..'))
# from utils.applyML_util import train_regression, eval_regression
# from utils.featureSelection_util import (pearson_correlation_fs, 
#                                          seleckKBest_fs, selectSequential_fs)

In [37]:
# the imports in this cell are required when running from Cloud (Colab/Kaggle)
# before running on cloud you nee to upload the .py files
# from 'Notebooks/utils' directory
from applyML_util import train_regression, eval_regression, showEvalutationGraph_regression
from featureSelection_util import (pearson_correlation_fs, 
                                   seleckKBest_fs, selectSequential_fs)

# SVC & SVR with Minimized Hyperparameters


**SVR Documentation link:** https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html

**GNB Documentation link:** https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html

**SK-Lego Documentation link:** https://scikit-lego.netlify.app/meta.html#Zero-Inflated-Regressor

In [38]:
!pip install sklego

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklego.meta import ZeroInflatedRegressor #!pip install sklego
from sklearn.svm import SVR
from sklearn.naive_bayes import GaussianNB

In [40]:
# global random seed
RAND_SEED = 42

ziReg = ZeroInflatedRegressor(classifier=GaussianNB(), regressor=SVR())

# initial model with only random seed and not any hyper-parametes
initial_model = ziReg

# hyper-parameters
regressor__kernel = ['poly']
regressor__degree = [5,6]
regressor__epsilon = [0.5]
regressor__C = [6,7,8,9]

classifier__var_smoothing = [1e-14,1e-13, 1e-12,1e-11, 1e-10, 1e-9, 1e-8, 1e-7]


param_grid = {'classifier__var_smoothing' : classifier__var_smoothing,
    'regressor__kernel': regressor__kernel,
    'regressor__C': regressor__C, 'regressor__epsilon': regressor__epsilon, 
    'regressor__degree': regressor__degree}

# # variables needed for showEvalGraph_regression() function
# MODEL_CLASS = ziReg
# x_axis_param_name = 'regressor__C' #???classifier__C
# x_axis_vals = regressor__C

In [41]:
ziReg.get_params().keys()


dict_keys(['classifier__priors', 'classifier__var_smoothing', 'classifier', 'regressor__C', 'regressor__cache_size', 'regressor__coef0', 'regressor__degree', 'regressor__epsilon', 'regressor__gamma', 'regressor__kernel', 'regressor__max_iter', 'regressor__shrinking', 'regressor__tol', 'regressor__verbose', 'regressor'])

## 1. Experimentation on the Weather Daily dataset

In [42]:
# Load the train dataset
weather_daily_train_df = pd.read_csv('https://raw.githubusercontent.com/ferdouszislam/Weather-WaterLevel-Prediction-ML/main/Datasets/brri-datasets/final-dataset/train/brri-weather_train_regression.csv')

# Load the test set
weather_daily_test_df = pd.read_csv('https://raw.githubusercontent.com/ferdouszislam/Weather-WaterLevel-Prediction-ML/main/Datasets/brri-datasets/final-dataset/test/brri-weather_test_regression.csv')

In [43]:
# train model
model, selected_hyperparams, train_r2, train_mae, train_rmse = train_regression(initial_model, param_grid, weather_daily_train_df, cls='Rainfall (mm)')
print(f'Selected hyperparameters: {selected_hyperparams}')
# performance on the train set
print(f'Train set performance: r2-score={train_r2}, mae={train_mae}, rmse={train_rmse}')

Selected hyperparameters: {'classifier__var_smoothing': 1e-14, 'regressor__C': 9, 'regressor__degree': 5, 'regressor__epsilon': 0.5, 'regressor__kernel': 'poly'}
Train set performance: r2-score=0.1951, mae=5.0889, rmse=13.4494


In [44]:
# # r2-scores graph on the train set

# # hyper-parameters selected by GridSearchCV
# selected_model_params = selected_hyperparams
# #selected_model_params['random_state'] = RAND_SEED

# showEvalutationGraph_regression(MODEL_CLASS, weather_daily_train_df, cls='Rainfall (mm)', 
#                                 x_axis_param_name=x_axis_param_name, x_axis_param_vals=x_axis_vals, 
#                                 selected_model_params=selected_model_params)

In [45]:
# test model
test_r2, test_mae, test_rmse = eval_regression(model, weather_daily_test_df, cls='Rainfall (mm)')
# performance on the test set
print(f'Test set performance: r2-score={test_r2}, mae={test_mae}, rmse={test_rmse}')

Test set performance: r2-score=0.1095, mae=5.4514, rmse=16.3167


### 1.1 Apply Pearson Feature Selection to Daily Weather Dataset

In [46]:
# select features from the train dataset
weather_daily_fs1_train_df, cols_to_drop = pearson_correlation_fs(weather_daily_train_df, 'Rainfall (mm)')

# keep only selected features on the test dataset
weather_daily_fs1_test_df = weather_daily_test_df.drop(columns=cols_to_drop)

dropping Sunshine (hour/day) from (Cloudy (hour/day), Sunshine (hour/day))


In [47]:
# train model
model, selected_hyperparams, train_r2, train_mae, train_rmse = train_regression(initial_model, param_grid, weather_daily_fs1_train_df, cls='Rainfall (mm)')
print(f'Selected hyperparameters: {selected_hyperparams}')
# performance on the train set
print(f'Train set performance: r2-score={train_r2}, mae={train_mae}, rmse={train_rmse}')

Selected hyperparameters: {'classifier__var_smoothing': 1e-14, 'regressor__C': 6, 'regressor__degree': 6, 'regressor__epsilon': 0.5, 'regressor__kernel': 'poly'}
Train set performance: r2-score=0.207, mae=5.0645, rmse=13.3449


In [48]:
# # r2-scores graph on the train set

# # hyper-parameters selected by GridSearchCV
# selected_model_params = selected_hyperparams
# #selected_model_params['random_state'] = RAND_SEED

# showEvalutationGraph_regression(MODEL_CLASS, weather_daily_fs1_train_df, cls='Rainfall (mm)', 
#                                 x_axis_param_name=x_axis_param_name, x_axis_param_vals=x_axis_vals, 
#                                 selected_model_params=selected_model_params)

In [49]:
# test model
test_r2, test_mae, test_rmse = eval_regression(model, weather_daily_fs1_test_df, cls='Rainfall (mm)')
# performance on the test set
print(f'Test set performance: r2-score={test_r2}, mae={test_mae}, rmse={test_rmse}')

Test set performance: r2-score=0.1159, mae=5.4722, rmse=16.2583


### 1.2 Apply SelectKBest Feature Selection to Daily Weather Dataset

In [50]:
# select features from the train dataset
weather_daily_fs2_train_df, cols_to_drop = seleckKBest_fs(weather_daily_train_df, 'Rainfall (mm)', is_regression=True)

print('features dropped:', cols_to_drop)

# keep only selected features on the test dataset
weather_daily_fs2_test_df = weather_daily_test_df.drop(columns=cols_to_drop)

features dropped: ['Max Temp. (degree Celcius)', 'Solar Radiation (cal/cm^2/day)']


In [51]:
# train model
model, selected_hyperparams, train_r2, train_mae, train_rmse = train_regression(initial_model, param_grid, weather_daily_fs2_train_df, cls='Rainfall (mm)')
print(f'Selected hyperparameters: {selected_hyperparams}')
# performance on the train set
print(f'Train set performance: r2-score={train_r2}, mae={train_mae}, rmse={train_rmse}')

Selected hyperparameters: {'classifier__var_smoothing': 1e-14, 'regressor__C': 6, 'regressor__degree': 6, 'regressor__epsilon': 0.5, 'regressor__kernel': 'poly'}
Train set performance: r2-score=0.1915, mae=5.0807, rmse=13.4844


In [52]:
# r2-scores graph on the train set

# # hyper-parameters selected by GridSearchCV
# selected_model_params = selected_hyperparams
# #selected_model_params['random_state'] = RAND_SEED

# showEvalutationGraph_regression(MODEL_CLASS, weather_daily_fs2_train_df, cls='Rainfall (mm)', 
#                                 x_axis_param_name=x_axis_param_name, x_axis_param_vals=x_axis_vals, 
#                                 selected_model_params=selected_model_params)

In [53]:
# test model
test_r2, test_mae, test_rmse = eval_regression(model, weather_daily_fs2_test_df, cls='Rainfall (mm)')
# performance on the test set
print(f'Test set performance: r2-score={test_r2}, mae={test_mae}, rmse={test_rmse}')

Test set performance: r2-score=0.115, mae=5.4873, rmse=16.2666


### 1.3 Apply SelectSequential Feature Selection to Daily Weather Dataset

In [54]:
# select features from the train dataset
weather_daily_fs3_train_df, cols_to_drop = selectSequential_fs(weather_daily_train_df, 'Rainfall (mm)', is_regression=True)

print('features dropped:', cols_to_drop)

# keep only selected features on the test dataset
weather_daily_fs3_test_df = weather_daily_test_df.drop(columns=cols_to_drop)

features dropped: ['Actual Evaporation (mm)', 'Relative Humidity (morning, %)']


In [55]:
# train model
model, selected_hyperparams, train_r2, train_mae, train_rmse = train_regression(initial_model, param_grid, weather_daily_fs3_train_df, cls='Rainfall (mm)')
print(f'Selected hyperparameters: {selected_hyperparams}')
# performance on the train set
print(f'Train set performance: r2-score={train_r2}, mae={train_mae}, rmse={train_rmse}')

Selected hyperparameters: {'classifier__var_smoothing': 1e-14, 'regressor__C': 9, 'regressor__degree': 5, 'regressor__epsilon': 0.5, 'regressor__kernel': 'poly'}
Train set performance: r2-score=0.1918, mae=5.1421, rmse=13.4788


In [56]:
# r2-scores graph on the train set

# # hyper-parameters selected by GridSearchCV
# selected_model_params = selected_hyperparams
# #selected_model_params['random_state'] = RAND_SEED

# showEvalutationGraph_regression(MODEL_CLASS, weather_daily_fs3_train_df, cls='Rainfall (mm)', 
#                                 x_axis_param_name=x_axis_param_name, x_axis_param_vals=x_axis_vals, 
#                                 selected_model_params=selected_model_params)

In [57]:
# test model
test_r2, test_mae, test_rmse = eval_regression(model, weather_daily_fs3_test_df, cls='Rainfall (mm)')
# performance on the test set
print(f'Test set performance: r2-score={test_r2}, mae={test_mae}, rmse={test_rmse}')

Test set performance: r2-score=0.0843, mae=5.5754, rmse=16.5458
